In [ ]:
cd ..

In [ ]:
import chess
import numpy as np
from chess import engine
from tensorflow.keras import models

from models import get_model
from scripts import bitboard, moves

In [ ]:
board = chess.Board()

In [ ]:
model = get_model()
model.load_weights("models/ser/v1/model.h5")

In [ ]:
simple_engine = engine.SimpleEngine.popen_uci("lib/stockfish/stockfish")

In [ ]:
pv, vh = model.predict(np.array((bitboard(board),)))

In [ ]:
np.array(moves)[np.argsort(np.squeeze(pv))[::-1]][:5]

In [ ]:
np.squeeze(vh)

In [ ]:
simple_engine.play(board, engine.Limit(time=.1), info=engine.INFO_SCORE)

In [ ]:
move = board.push_uci("g4c8")
# board.pop()
board